In [0]:
import torchvision
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import albumentations as A
import cv2
from torchsummary import summary
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR
from albumentations import Compose, Normalize, HorizontalFlip, Cutout, Rotate, RandomCrop, PadIfNeeded
from albumentations.pytorch import ToTensor

In [0]:
import train_test
import Data_Loader

In [3]:

import Misclassified_image as MC
from TrainTestandUtils import TrainTestandUtils as Tt
import cyclic_lr as clr
from Albumentations import AlbumentationTransforms
from GradCAM import GradCAM
from LR_range_test import LR_test
from lr_finder import LRFinder
from Utils import *
from tinyimagenet import TinyImageNetDataSet


  pip install torch-lr-finder -v --global-option="amp"


In [4]:
import albumentations as A
import cv2
from albumentations import Compose
channel_means = (0.5, 0.5, 0.5)
channel_stdevs = (0.5, 0.5, 0.5)
train_transform = AlbumentationTransforms([       
                                                                
                                      A.HorizontalFlip(p = 0.7),
                                      A.PadIfNeeded(min_height=70, min_width=70, border_mode=4, value=None, always_apply=False, p=1.0),
                                      A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
                                      A.RandomCrop(64, 64, always_apply=False, p=1.0),
                                      A.VerticalFlip(always_apply=False, p=0.5),
                                      A.Rotate(limit=50, interpolation=1, border_mode=4, always_apply=False, p=0.5),
                                      A.Normalize(mean=channel_means, std=channel_stdevs),
                                      A.Cutout(num_holes=1, max_h_size=32,max_w_size = 32,p=0.7) 
                                       ])
test_transform = AlbumentationTransforms([A.Normalize(mean=channel_means, std=channel_stdevs)])
train_dataset , test_dataset,classes = TinyImageNetDataSet(train_split = 70,test_transforms = test_transform,train_transforms = train_transform)

Images already downloaded...


In [0]:

final_train , final_test = Data_Loader.load(train_dataset,test_dataset,batch_size=256)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (device)

cuda


In [0]:
from ResnetModel import ResNet18

model = ResNet18(num_classes=200)

In [8]:
new_model = model.to(device)
summary(new_model, input_size=(3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,728
       BatchNorm2d-2           [-1, 64, 64, 64]             128
            Conv2d-3           [-1, 64, 64, 64]          36,864
       BatchNorm2d-4           [-1, 64, 64, 64]             128
            Conv2d-5           [-1, 64, 64, 64]          36,864
       BatchNorm2d-6           [-1, 64, 64, 64]             128
        BasicBlock-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
           Conv2d-10           [-1, 64, 64, 64]          36,864
      BatchNorm2d-11           [-1, 64, 64, 64]             128
       BasicBlock-12           [-1, 64, 64, 64]               0
           Conv2d-13          [-1, 128, 32, 32]          73,728
      BatchNorm2d-14          [-1, 128,

In [0]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR,StepLR

optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9,weight_decay = 0.0001,nesterov = True ) 
criterion = nn.CrossEntropyLoss()
scheduler = OneCycleLR(optimizer, max_lr = 0.02, total_steps=None, epochs=50, 
                       steps_per_epoch=len(final_train), pct_start=1/3, 
                       anneal_strategy='linear', cycle_momentum=True, base_momentum=0.85, 
                       max_momentum=0.95, div_factor=10.0,final_div_factor =10)

In [0]:
train_test.train_model(model,device,final_train,final_test,optimizer,criterion,50,scheduler,batch_scheduler=True)

---------------------------------
EPOCH: 1 LR: 0.002




Test set: Average loss: 0.0176, Accuracy: 2491/33000 (7.55%)

---------------------------------
EPOCH: 2 LR: 0.0030802153253140165




Test set: Average loss: 0.0159, Accuracy: 4052/33000 (12.28%)

---------------------------------
EPOCH: 3 LR: 0.004160430650628033




Test set: Average loss: 0.0151, Accuracy: 5180/33000 (15.70%)

---------------------------------
EPOCH: 4 LR: 0.00524064597594205




Test set: Average loss: 0.0147, Accuracy: 5932/33000 (17.98%)

---------------------------------
EPOCH: 5 LR: 0.006320861301256066


...



Test set: Average loss: 0.0084, Accuracy: 16236/33000 (49.20%)

---------------------------------
EPOCH: 38 LR: 0.007920026578073088




Test set: Average loss: 0.0088, Accuracy: 15914/33000 (48.22%)

---------------------------------
EPOCH: 39 LR: 0.007326026578073087




Test set: Average loss: 0.0085, Accuracy: 16120/33000 (48.85%)

---------------------------------
EPOCH: 40 LR: 0.00673202657807309




Test set: Average loss: 0.0082, Accuracy: 16757/33000 (50.78%)

---------------------------------
EPOCH: 41 LR: 0.006138026578073089


Buffered data was truncated after reaching the output size limit.